**Aqui estan todas las funciones del proyecto por separado**

In [1]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime
import locale
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("dataset/dataset_cleaned.csv",low_memory=False)
data = pd.read_csv('dataset/dataset_ML.csv', low_memory=False) 

In [3]:
def peliculas_mes(mes):
    aux = df.groupby("month_name").size()
    respuesta = aux [mes]

    return {'mes':mes, 'cantidad':respuesta}

In [4]:
def peliculas_dia(dia):
    aux = df.groupby("day_name").size()
    respuesta = aux [dia]
    return {'dia':dia, 'cantidad':respuesta}

In [5]:
def franquicia(franquicia):
    aux = df.groupby("belongs_to_collection").size()
    cantidad = aux [franquicia]
    ganancia = df.query("belongs_to_collection == @franquicia")['revenue'].sum()
    promedio = df.query("belongs_to_collection == @franquicia")['revenue'].mean()
    return {'franquicia':franquicia, 'cantidad de peliculas':cantidad, 'ganancia_total':ganancia, 'ganancia_promedio':promedio}

In [6]:
def peliculas_pais(pais):
    df["production_countries"] = df["production_countries"].fillna("")
    df_filtrado = df[df['production_countries'].apply(lambda x: pais in str(x))]
    cantidad_peliculas = len(df_filtrado)
    return {'pais':pais, 'cantidad':cantidad_peliculas}

In [7]:
def productoras(productora):
    df_filtrado = df[df['production_companies'].apply(lambda x: productora in str(x))]
    cantidad = len(df_filtrado)
    ganancia= df_filtrado["revenue"].sum()
    return {'productora':productora, 'ganancia_total':ganancia, 'cantidad':cantidad}

In [8]:
def retorno(pelicula):
    df_filtrado = df[df['title'].apply(lambda x: pelicula.strip() == str(x).strip())]
    inversion= df_filtrado["budget"].sum()
    ganancia= df_filtrado["revenue"].sum()
    retorno= df_filtrado["return"].sum()
    anio = df_filtrado["release_date"]
    return {'pelicula':pelicula, 'inversion':inversion, 'ganacia':ganancia,'retorno':retorno, 'anio':anio}

In [ ]:
# FUNCIÓN DEL Ml
def get_recommendations(movie_name, top_n=5):
    top_n = 5
    n = 5000
    data_subset = data.head(n)
    X = data_subset[['belongs_to_collection', 'genres', 'original_language', "popularity", "production_companies", "release_date", "runtime", "status", "vote_average", "return"]]  
    similarity_matrix = cosine_similarity(X)
    movie_index = data[data['title'] == movie_name].index[0] 
    movie_similarities = similarity_matrix[movie_index]
    top_indices = movie_similarities.argsort()[-top_n-1:-1][::-1]  
    recommendations = data.loc[top_indices,"title"] 
    return recommendations["title"]